In [3]:
import import_ipynb

from load_data import train_data
from load_data import song_meta
from load_data import genre_gn_all

importing Jupyter notebook from load_data.ipynb
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115071 non-null  object
 1   id            115071 non-null  int64 
 2   plylst_title  115071 non-null  object
 3   songs         115071 non-null  object
 4   like_cnt      115071 non-null  int64 
 5   updt_date     115071 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  707989 non-null  object
 1   issue_date              707989 non-null  int64 
 2   album_name              707985 non-null  object
 3   album_id                707989 non-null  int64 
 4   artist_id_baske

In [4]:
# train_data.head()

In [5]:
# train_data_copy = train_data.copy()

In [6]:
# pivot_table = [[0]*115071 for i in range(707988)]

# for user in train_data.index:
#     for song in train_data['songs'][user]:
#         pivot_table[user][song]=1;
            
# pivot_table

In [7]:
# pivot_table = [[0]*15050 for i in range(1634)]

# for user in range(100):
#     for song in train_data['songs'][user]:
#         pivot_table[user][song]=1;
            
# for i in pivot_table :
#     for j in i:
#         print(j,end=" ")
#     print()

In [8]:
# def preprocessed(data):
#     n_data = len(data)
#     data['nid'] = range(n_data) # tain_data id 새로 생성 (load_data에서 확인해본 결과 0 - 115070가 아니여서)
#     data_song = data['songs']
#     song_counter = Counter([sg for sgs in data_song for sg in sgs])
#     song_dict = {x: song_counter[x] for x in song_counter}

#     song_id_sid = dict()
#     song_sid_id = dict()
#     for i, t in enumerate(song_dict):
#       song_id_sid[t] = i
#       song_sid_id[i] = t

#     n_songs = len(song_dict)
    
#     data['songs_id'] = data['songs'].map(lambda x: [song_id_sid.get(s) for s in x if song_id_sid.get(s) != None])
    
#     data_use = data[['nid','updt_date','songs_id']]
#     data_use.loc[:,'num_songs'] = data_use['songs_id'].map(len) # num_songs의 열을 만들어서 플리에 들어있는 song의 수 저장
#     data_use = data_use.set_index('nid') # nid를 id로 사용!
    
#     return data_use

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import import_ipynb
import json
import distutils.dir_util
import io
import os
import sys
plt.style.use('seaborn')
sns.set(font_scale = 2.5)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [10]:
playlist = []
for index in train_data.index:
    tag = train_data['plylst_title'][index]
    if('크리스마스' in tag or '캐럴' in tag or '캐롤' in tag):
        playlist.append(index)

playlist

[3,
 39,
 82,
 172,
 241,
 252,
 264,
 396,
 411,
 576,
 603,
 817,
 827,
 903,
 913,
 991,
 999,
 1103,
 1220,
 1272,
 1340,
 1382,
 1389,
 1408,
 1412,
 1440,
 1496,
 1513,
 1569,
 1645,
 1972,
 2010,
 2069,
 2091,
 2136,
 2182,
 2358,
 2584,
 2800,
 2806,
 2824,
 2857,
 2861,
 3017,
 3060,
 3138,
 3196,
 3264,
 3277,
 3409,
 3455,
 3501,
 3545,
 3564,
 3568,
 3607,
 3791,
 3802,
 3839,
 3912,
 3960,
 3974,
 3986,
 4054,
 4076,
 4129,
 4142,
 4314,
 4354,
 4364,
 4400,
 4403,
 4507,
 4510,
 4555,
 4613,
 4699,
 4744,
 4778,
 5060,
 5120,
 5123,
 5166,
 5256,
 5269,
 5374,
 5412,
 5426,
 5431,
 5536,
 5560,
 5669,
 5673,
 5741,
 5767,
 5807,
 6005,
 6181,
 6202,
 6369,
 6624,
 6687,
 6731,
 6748,
 6800,
 6872,
 6886,
 6927,
 6989,
 7015,
 7043,
 7059,
 7187,
 7350,
 7368,
 7377,
 7399,
 7485,
 7495,
 7533,
 7704,
 7733,
 7749,
 7835,
 7846,
 8006,
 8010,
 8025,
 8048,
 8116,
 8224,
 8227,
 8240,
 8258,
 8343,
 8403,
 8502,
 8513,
 8528,
 8610,
 8655,
 8802,
 8837,
 8841,
 8894,
 9133,

In [11]:
i=0
playlist_train = pd.DataFrame([], columns=["Index", "tags", "id", "plylst_title", "songs", "like_cnt", "updt_date"])
for row in train_data.itertuples():
    if row.Index in playlist :
        playlist_train.loc[i] = row
        i+= 1

playlist_train

,Index,tags,id,plylst_title,songs,like_cnt,updt_date
0,3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
1,39,"[겨울, 크리스마스, 캐롤]",43665,크리스마스를 책임질 캐롤 모음,"[273672, 358186, 657321, 610130, 387284, 51860...",1,2018-12-01 10:12:41.000
2,82,"[분위기좋은, 크리스마스]",60192,느낌있는 크리스마스 쏭,"[187632, 432118, 557519, 423058, 387647, 61771...",1,2019-12-03 20:33:17.000
3,172,"[휴식, 기분전환]",101383,크리스마스9,"[405485, 381653, 566122, 546313, 158539, 68697...",0,2019-11-18 17:10:36.000
4,241,[겨울],43159,미리 메리 크리스마스,"[326204, 174967, 117169, 703323, 638488, 59252...",17,2015-11-22 01:28:30.000
...,...,...,...,...,...,...,...
1629,114710,"[겨울, 감성, 밤, 카페, 새벽, RnB, Pop, 캐롤, 추위, Malcontent]",97036,캐롤이 지겨워진 당신을 위한 99곡,"[262702, 34548, 274504, 523032, 590194, 85226,...",440,2017-01-02 07:43:46.000
1630,114716,"[기분좋은, 두근두근, 봄, 취향저격, 벚꽃, 봄캐롤, 설렘]",14310,봄이 오면 찾게 돼! 두근두근 기분 좋은 봄캐럴,"[411161, 132247, 109977, 113847, 159117, 41555...",4802,2020-04-07 17:31:10.000
1631,114947,"[기분전환, 잔잔한]",21781,크리스마스441,"[405485, 381653, 566122, 546313, 158539, 68697...",1,2019-11-19 15:30:30.000
1632,115004,[OST],140663,커플의 러브엑츄얼리 vs 솔로의 나홀로집에 크리스마스 대표 영화 OST,"[628409, 63129, 50450, 387546, 186956, 707422,...",2,2015-12-17 22:46:14.000


In [19]:
cnt=0;
for i in range(len(playlist_train)):
    if len(playlist_train.songs[i]) < 5:
        cnt+=1

print(cnt)

15


In [13]:
# 캐롤 리스트에 캐롤 추가해주기- 한번만 돌리기
carol_list = []
for index in playlist:  # 크리스마스 플레이리스트 번호 돌기
    for song in train_data['songs'][index]:
        if song not in carol_list:
            carol_list.append(song)

0


In [14]:
i=0
carol_data = pd.DataFrame([], columns=["Index","song_gn_dtl_gnr_basket","issue_date","album_name","album_id","artist_id_basket","song_name","song_gn_gnr_basket","artist_name_basket", "id"])
for row in song_meta.itertuples():
    if row.Index in carol_list :
        carol_data.loc[i] = row
        i += 1

carol_data

,Index,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,23,"[GN1601, GN1615, GN1610]",20171006,Christmas at Westminster Abbey,10104201,"[345328, 36652, 347399]",Anonymous: Up! Awake! From Highest Steeple - A...,[GN1600],"[Choir Of Westminster Abbey, Simon Preston, Ch...",23
1,41,"[GN1701, GN1703, GN1702]",20121109,Christmas (Deluxe Special Edition),2167838,[35370],Mis Deseos / Feliz Navidad (with Thalia),[GN1700],[Michael Buble],41
2,54,[GN0101],20190408,있어줄래,10271322,[500421],있어줄래,[GN0100],[길구봉구],54
3,70,"[GN0105, GN0101]",20121204,WINTER POETRY,2169041,[1741],너에게 전하지 못한 말,[GN0100],[신혜성],70
4,71,"[GN1304, GN1301]",20110810,Simple Steps,2058451,[614091],Rollercoaster,[GN1300],[Sam Ock],71
...,...,...,...,...,...,...,...,...,...,...
15045,707796,[GN1701],20111207,Christmas Jazz Music,2233676,[757383],O Christmas Tree (Christmas Songs For Family),[GN1700],[Christmas Jazz],707796
15046,707821,[GN0901],20110110,A Kylie Christmas,159204,[2160],Santa Baby,[GN0900],[Kylie Minogue],707821
15047,707913,[GN0101],20131031,Always be there,2213566,[107728],Always be there,[GN0100],[브라운 아이드 소울],707913
15048,707969,"[GN0401, GN0402]",20110902,I Decide (Piano Remix),2010118,[202407],I Decide (Piano Remix),[GN0400],[헤리티지(Heritage)],707969


In [15]:
playlist_train.to_json(orient = 'records')

'[{"Index":3,"tags":["\\uc5f0\\ub9d0","\\ub208\\uc624\\ub294\\ub0a0","\\uce90\\ub7f4","\\ubd84\\uc704\\uae30","\\ub530\\ub4ef\\ud55c","\\ud06c\\ub9ac\\uc2a4\\ub9c8\\uc2a4\\uce90\\ub7f4","\\uaca8\\uc6b8\\ub178\\ub798","\\ud06c\\ub9ac\\uc2a4\\ub9c8\\uc2a4","\\uaca8\\uc6b8\\uc655\\uad6d","\\ud06c\\ub9ac\\uc2a4\\ub9c8\\uc2a4\\uc1a1"],"id":147456,"plylst_title":"\\ud06c\\ub9ac\\uc2a4\\ub9c8\\uc2a4 \\ubd84\\uc704\\uae30\\uc5d0 \\ud760\\ubed1 \\ucde8\\ud558\\uace0 \\uc2f6\\uc744\\ub54c","songs":[394031,195524,540149,287984,440773,100335,556301,655561,534818,695032,516602,521739,97057,703323,295250,25155,24275,273672,334095,284990,679582,664342,637135,68528,243754,417111,414226,338078,384299,542735,457519,453762,349398,631142,406082,548389,205179,567076],"like_cnt":33,"updt_date":"2019-12-05 15:15:18.000"},{"Index":39,"tags":["\\uaca8\\uc6b8","\\ud06c\\ub9ac\\uc2a4\\ub9c8\\uc2a4","\\uce90\\ub864"],"id":43665,"plylst_title":"\\ud06c\\ub9ac\\uc2a4\\ub9c8\\uc2a4\\ub97c \\ucc45\\uc784\\uc9c8 \\uce

In [16]:
from util import write_json, load_json

write_json(playlist_train.to_json(orient = 'records'), "playlist_train.json")
write_json(carol_data.to_json(orient = 'records'), "carol_data.json")

importing Jupyter notebook from util.ipynb


In [17]:
import pandas as pd

test_playlist = load_json("data/playlist_train.json")
test_carol = load_json("data/carol_data.json")

In [18]:
df = pd.DataFrame(eval(test_playlist))
df

,Index,tags,id,plylst_title,songs,like_cnt,updt_date
0,3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
1,39,"[겨울, 크리스마스, 캐롤]",43665,크리스마스를 책임질 캐롤 모음,"[273672, 358186, 657321, 610130, 387284, 51860...",1,2018-12-01 10:12:41.000
2,82,"[분위기좋은, 크리스마스]",60192,느낌있는 크리스마스 쏭,"[187632, 432118, 557519, 423058, 387647, 61771...",1,2019-12-03 20:33:17.000
3,172,"[휴식, 기분전환]",101383,크리스마스9,"[405485, 381653, 566122, 546313, 158539, 68697...",0,2019-11-18 17:10:36.000
4,241,[겨울],43159,미리 메리 크리스마스,"[326204, 174967, 117169, 703323, 638488, 59252...",17,2015-11-22 01:28:30.000
...,...,...,...,...,...,...,...
1629,114710,"[겨울, 감성, 밤, 카페, 새벽, RnB, Pop, 캐롤, 추위, Malcontent]",97036,캐롤이 지겨워진 당신을 위한 99곡,"[262702, 34548, 274504, 523032, 590194, 85226,...",440,2017-01-02 07:43:46.000
1630,114716,"[기분좋은, 두근두근, 봄, 취향저격, 벚꽃, 봄캐롤, 설렘]",14310,봄이 오면 찾게 돼! 두근두근 기분 좋은 봄캐럴,"[411161, 132247, 109977, 113847, 159117, 41555...",4802,2020-04-07 17:31:10.000
1631,114947,"[기분전환, 잔잔한]",21781,크리스마스441,"[405485, 381653, 566122, 546313, 158539, 68697...",1,2019-11-19 15:30:30.000
1632,115004,[OST],140663,커플의 러브엑츄얼리 vs 솔로의 나홀로집에 크리스마스 대표 영화 OST,"[628409, 63129, 50450, 387546, 186956, 707422,...",2,2015-12-17 22:46:14.000
